# Generacion de texto utilizando LLM para Google Colab

Este notebook utiliza [https://github.com/oobabooga/text-generation-webui](https://github.com/oobabooga/text-generation-webui) para lanzar modelos conversacionales en modo chat.

Lanza todas las celdas y al final del documento aparecera una URL de gradio que hace de tunel a la maquina local

## Parametros

* **save_logs_to_google_drive**: Guarda los logs en google drive para que persistan entre sesiones.
* **text_streaming**: Permite que el texto aparezca en tiempo real en lugar de esperar a que se haya generado todo resultado.
* **activate_sending_pictures**: Añade un menu para enviar imagenes al bot que automaticamente seran evaluadas utilizando BLIP.


## Creditos

Basado en [notebook original por 81300](https://colab.research.google.com/github/81300/AI-Notebooks/blob/main/Colab-TextGen-GPU.ipynb).


In [ ]:
!nvidia-smi

In [ ]:
#@title 1. Manten esta pestaña activa para prevenir que Colab te desconecte { display-mode: "form" }

#@markdown Presiona el boton de play que aparece a continuacion:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
#@title 2. Instala la UI web
#@markdown Ten en cuenta que estos modelos son pesados y la capa gratuita de GDrive solo cuenta con 15gb <br>
save_logs_to_google_drive = True #@param {type:"boolean"} 
save_everything_to_google_drive = True #@param {type:"boolean"} 
install_gptq = True
from IPython.display import clear_output
if save_logs_to_google_drive:
  import os
  import shutil
  from google.colab import drive
  drive.mount('/content/drive')
  base_folder = '/content/drive/MyDrive'

if save_everything_to_google_drive:
    import os
    import shutil
    from google.colab import drive
    drive.mount('/content/drive')
    base_folder = '/content/drive/MyDrive'
    repo_dir = '/content/drive/MyDrive/text-generation-webui'
    model_dir = '/content/drive/MyDrive/text-generation-webui/models'
    gptq_dir = '/content/drive/MyDrive/text-generation-webui/repositories/GPTQ-for-LLaMa'
    if os.path.exists(repo_dir):
        %cd {repo_dir}
        !git pull
    else:
        %cd /content/drive/MyDrive/
        !git clone https://github.com/oobabooga/text-generation-webui

else:
    model_dir = '/content/text-generation-webui/models'
    repo_dir = '/content/text-generation-webui'
    %cd /content
    !git clone https://github.com/oobabooga/text-generation-webui



if save_logs_to_google_drive:
  if not os.path.exists(f"{base_folder}/oobabooga-data"):
    os.mkdir(f"{base_folder}/oobabooga-data")
  if not os.path.exists(f"{base_folder}/oobabooga-data/logs"):
    os.mkdir(f"{base_folder}/oobabooga-data/logs")
  if not os.path.exists(f"{base_folder}/oobabooga-data/softprompts"):
    os.mkdir(f"{base_folder}/oobabooga-data/softprompts")
  if not os.path.exists(f"{base_folder}/oobabooga-data/characters"):
    shutil.move("text-generation-webui/characters", f"{base_folder}/oobabooga-data/characters")
  else:
    !rm -r "text-generation-webui/characters"
    
  !rm -r "text-generation-webui/softprompts"
  !ln -s "$base_folder/oobabooga-data/logs" "text-generation-webui/logs"
  !ln -s "$base_folder/oobabooga-data/softprompts" "text-generation-webui/softprompts"
  !ln -s "$base_folder/oobabooga-data/characters" "text-generation-webui/characters"

else:
  !mkdir text-generation-webui/logs

!ln -s text-generation-webui/logs .
!ln -s text-generation-webui/characters .
!ln -s text-generation-webui/models .
%rm -r sample_data
%cd text-generation-webui
!wget https://raw.githubusercontent.com/pcrii/Philo-Colab-Collection/main/settings-colab-template.json -O settings-colab-template.json

# if not os.path.exists(f"{repo_dir}/autograd_4bit.py"):
#   !wget https://raw.githubusercontent.com/johnsmith0031/alpaca_lora_4bit/main/autograd_4bit.py -O autograd_4bit.py

# if not os.path.exists(f"{repo_dir}/amp_wrapper.py"):
#   !wget https://raw.githubusercontent.com/johnsmith0031/alpaca_lora_4bit/main/amp_wrapper.py -O amp_wrapper.py

# Install requirements
!pip install -r requirements.txt --upgrade
!pip install -r extensions/google_translate/requirements.txt
!pip install -r extensions/silero_tts/requirements.txt
print(f"\033[1;32;1m\n --> If you see a warning about \"pydevd_plugins\", just ignore it and move on to Step 3. There is no need to restart the runtime.\n\033[0;37;0m")

if install_gptq:
    if save_everything_to_google_drive:
        if os.path.exists(gptq_dir):
            %cd {gptq_dir}
            !git pull
            !pip install ninja
            !pip install -r requirements.txt
            !python setup_cuda.py install

        else:
            !mkdir /content/drive/MyDrive/text-generation-webui/repositories
            %cd /content/drive/MyDrive/text-generation-webui/repositories
            !git clone https://github.com/oobabooga/GPTQ-for-LLaMa.git -b cuda
            %cd GPTQ-for-LLaMa
            !pip install ninja
            !pip install -r requirements.txt
            !python setup_cuda.py install
    else:
        %mkdir /content/text-generation-webui/repositories/
        %cd /content/text-generation-webui/repositories/
        !git clone https://github.com/oobabooga/GPTQ-for-LLaMa.git -b cuda
        %cd GPTQ-for-LLaMa
        !pip install ninja
        !pip install -r requirements.txt
        !python setup_cuda.py install


if not os.path.exists(f"{repo_dir}/repositories"):
  os.mkdir(f"{repo_dir}/repositories")


if os.path.exists(f"{repo_dir}/repositories/alpaca_lora_4bit"):
  %cd {repo_dir}/repositories/alpaca_lora_4bit
  !git pull
  !pip install -r requirements.txt
else:
  %cd {repo_dir}/repositories/
  !git clone https://github.com/johnsmith0031/alpaca_lora_4bit
  %cd alpaca_lora_4bit
  !pip install -r requirements.txt

!pip install git+https://github.com/sterlind/GPTQ-for-LLaMa.git@lora_4bit

# clear_output()
# print("Finished")

In [ ]:
#@title Descarga el modelo
#@markdown Puedes insertar cualquier modelo de huggingface con el formato Organizacion/modelo
model_download = "anon8231489123/vicuna-13b-GPTQ-4bit-128g" #@param ["anon8231489123/gpt4-x-alpaca-13b-native-4bit-128g", "anon8231489123/vicuna-13b-GPTQ-4bit-128g"] {allow-input: true}
%cd {repo_dir}
!python download-model.py {model_download}
!rm {model_dir}/place-your-models-here.txt
clear_output()
if save_logs_to_google_drive or save_everything_to_google_drive:
    drive_NOT_mounted = False
else:
    drive_NOT_mounted = True

if drive_NOT_mounted:
  import os
print("Available Models")
print(os.listdir(model_dir))



In [ ]:
#@title 3. Inicia la interfaz

import json

model_load = "anon8231489123_vicuna-13b-GPTQ-4bit-128g" #@param {type:"string"}
# Parameters
load_4bit_models = True #@param {type:"boolean"}

groupsize_128 = True 
load_in_8bit = False 
chat = False 

text_streaming = True #@param {type:"boolean"}
activate_silero_text_to_speech = False
activate_sending_pictures = False #@param {type:"boolean"}
activate_character_bias = False
chat_language = "English" 


activate_google_translate = (chat_language != "English")

language_codes = {'Afrikaans': 'af', 'Albanian': 'sq', 'Amharic': 'am', 'Arabic': 'ar', 'Armenian': 'hy', 'Azerbaijani': 'az', 'Basque': 'eu', 'Belarusian': 'be', 'Bengali': 'bn', 'Bosnian': 'bs', 'Bulgarian': 'bg', 'Catalan': 'ca', 'Cebuano': 'ceb', 'Chinese (Simplified)': 'zh-CN', 'Chinese (Traditional)': 'zh-TW', 'Corsican': 'co', 'Croatian': 'hr', 'Czech': 'cs', 'Danish': 'da', 'Dutch': 'nl', 'English': 'en', 'Esperanto': 'eo', 'Estonian': 'et', 'Finnish': 'fi', 'French': 'fr', 'Frisian': 'fy', 'Galician': 'gl', 'Georgian': 'ka', 'German': 'de', 'Greek': 'el', 'Gujarati': 'gu', 'Haitian Creole': 'ht', 'Hausa': 'ha', 'Hawaiian': 'haw', 'Hebrew': 'iw', 'Hindi': 'hi', 'Hmong': 'hmn', 'Hungarian': 'hu', 'Icelandic': 'is', 'Igbo': 'ig', 'Indonesian': 'id', 'Irish': 'ga', 'Italian': 'it', 'Japanese': 'ja', 'Javanese': 'jw', 'Kannada': 'kn', 'Kazakh': 'kk', 'Khmer': 'km', 'Korean': 'ko', 'Kurdish': 'ku', 'Kyrgyz': 'ky', 'Lao': 'lo', 'Latin': 'la', 'Latvian': 'lv', 'Lithuanian': 'lt', 'Luxembourgish': 'lb', 'Macedonian': 'mk', 'Malagasy': 'mg', 'Malay': 'ms', 'Malayalam': 'ml', 'Maltese': 'mt', 'Maori': 'mi', 'Marathi': 'mr', 'Mongolian': 'mn', 'Myanmar (Burmese)': 'my', 'Nepali': 'ne', 'Norwegian': 'no', 'Nyanja (Chichewa)': 'ny', 'Pashto': 'ps', 'Persian': 'fa', 'Polish': 'pl', 'Portuguese (Portugal, Brazil)': 'pt', 'Punjabi': 'pa', 'Romanian': 'ro', 'Russian': 'ru', 'Samoan': 'sm', 'Scots Gaelic': 'gd', 'Serbian': 'sr', 'Sesotho': 'st', 'Shona': 'sn', 'Sindhi': 'sd', 'Sinhala (Sinhalese)': 'si', 'Slovak': 'sk', 'Slovenian': 'sl', 'Somali': 'so', 'Spanish': 'es', 'Sundanese': 'su', 'Swahili': 'sw', 'Swedish': 'sv', 'Tagalog (Filipino)': 'tl', 'Tajik': 'tg', 'Tamil': 'ta', 'Telugu': 'te', 'Thai': 'th', 'Turkish': 'tr', 'Ukrainian': 'uk', 'Urdu': 'ur', 'Uzbek': 'uz', 'Vietnamese': 'vi', 'Welsh': 'cy', 'Xhosa': 'xh', 'Yiddish': 'yi', 'Yoruba': 'yo', 'Zulu': 'zu'}


%cd {repo_dir}
# Applying the selected language and setting the prompt size to 2048
# if 8bit mode is selected
j = json.loads(open('settings-colab-template.json', 'r').read())
j["google_translate-language string"] = language_codes[chat_language]
if load_in_8bit:
  j["chat_prompt_size"] = 2048
with open('settings-colab.json', 'w') as f:
  f.write(json.dumps(j, indent=4))

params = set()
if chat:
  params.add('--cai-chat')

if load_in_8bit:
  params.add('--load-in-8bit')
#if auto_devices:
#  params.add('--auto-devices')
if load_4bit_models:
  params.add('--wbits 4')

if groupsize_128:
  params.add('--groupsize 128')

active_extensions = []
if activate_sending_pictures:
  active_extensions.append('send_pictures')
if activate_character_bias:
  active_extensions.append('character_bias')
if activate_google_translate:
  active_extensions.append('google_translate')
if activate_silero_text_to_speech:
  active_extensions.append('silero_tts')

if len(active_extensions) > 0:
  params.add(f'--extensions {" ".join(active_extensions)}')

if not text_streaming or activate_google_translate or activate_silero_text_to_speech:
  params.add('--no-stream')
if activate_character_bias:
  params.add('--verbose')

params.add('--monkey-patch')

# Starting the web UI
cmd = f"python server.py --share --model {model_load} --settings settings-colab.json {' '.join(params)}"
print(cmd)
!$cmd